# Use Pre-Trained Model

In [4]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import torch
import tomli
import matplotlib.pyplot as plt
from os import environ
from pathlib import Path
from buildings_bench import load_torch_dataset
from buildings_bench.models import model_factory

In [6]:
class DataHandler:
    def __init__(self): pass

    def load_dataset(self, dataset_name, scaler_transform):
        dataset = load_torch_dataset(
            name=dataset_name,
            apply_scaler_transform=scaler_transform,
            scaler_transform_path=Path(environ["TRANSFORM_PATH"])
        )
        return dataset

    def create_dataloader(self, building_dataset):
        dataloader = torch.utils.data.DataLoader(
                        building_dataset,
                        batch_size=360,
                        shuffle=False)
        return dataloader

In [7]:
def main(dataset_name, scaler_transform, device):
    handler = DataHandler()
    dataset = handler.load_dataset(dataset_name, scaler_transform)    

    target_list = []
    prediction_list = []
    
    for building_id, building_dataset in dataset:
        
        inverse_transform = building_dataset.datasets[0].load_transform.undo_transform
        dataloader = handler.create_dataloader(building_dataset)
        
        for batch in dataloader:
            for key, value in batch.items():
                batch[key] = value.to(device)

            with torch.cuda.amp.autocast(enabled=(device == 'cuda:0')):
                predictions = model(batch)
        
            targets = batch['load'][:, context_len:]
            targets = inverse_transform(targets)
            target_list.append(targets.detach().cpu())
            
            predictions = inverse_transform(predictions)
            prediction_list.append(predictions.detach().cpu())

    prediction_list = torch.cat(prediction_list)
    target_list = torch.cat(target_list)
    
    return prediction_list, target_list

In [9]:
if __name__ == "__main__":

    DATASET = 'electricity'
    DEVICE = 'cuda:0'
    MODEL = 'MyModel'
    SCALER_TRANSFORM = 'boxcox'

    PATH = '/pscratch/sd/n/nrushad'
    environ["PATH"] = PATH
    
    environ["REPO_PATH"] = f'{PATH}/BuildingsBenchTutorial/BuildingsBench/'
    environ["CONFIG_PATH"] = f'{PATH}/BuildingsBenchTutorial/BuildingsBench/buildings_bench/configs/{MODEL}.toml'
    environ["BUILDINGS_BENCH"] = f'{PATH}/Dataset'
    environ["TRANSFORM_PATH"] = f'{PATH}/Dataset/metadata/transforms'
    
    prediction_list, target_list = main(DATASET, SCALER_TRANSFORM, DEVICE)
    # plt.plot(prediction_list[0].squeeze())
    # plt.plot(target_list[0].squeeze())

TypeError: TimeSeriesTransformerForPrediction.forward() missing 2 required positional arguments: 'past_time_features' and 'past_observed_mask'